In [1]:
from surprise import Dataset, Reader
import pandas as pd

In [2]:
path = '/home/jimmy/CILProject22/data/raw/cv10'

In [3]:
reader = Reader(line_format = 'user item rating', sep='\t')

In [4]:
train_path = '/home/jimmy/CILProject22/data/raw/cv10/train_split_'
val_path = '/home/jimmy/CILProject22/data/raw/cv10/test_split_'
def preprocess(file_path, splits = 10):
    for i in range(splits):
        print(f"Converting split {i}")
        df = pd.read_csv(file_path+f'{i}.csv')
        indices_users, indices_movies = [], []
        values = []
        for j, x in df.iterrows():
            name, val = x['Id'], x['Prediction']
            user, movie = name.replace('c', '').replace('r', '').split('_')
            indices_users.append(user)
            indices_movies.append(movie)
            values.append(val)
        data = {'user': indices_users, 'movies': indices_movies, 'values': values}
        df = pd.DataFrame(data)
        df.to_csv(file_path+f'{i}_converted.csv', index=False)

In [5]:
preprocess(train_path)

Converting split 0
Converting split 1
Converting split 2
Converting split 3
Converting split 4
Converting split 5
Converting split 6
Converting split 7
Converting split 8
Converting split 9


In [6]:
preprocess(val_path)

Converting split 0
Converting split 1
Converting split 2
Converting split 3
Converting split 4
Converting split 5
Converting split 6
Converting split 7
Converting split 8
Converting split 9


In [9]:
train_path = '/home/jimmy/CILProject22/data/raw/cv10/train_split_'
val_path = '/home/jimmy/CILProject22/data/raw/cv10/test_split_'
def load_dataset(file_dir, split = 0):
    df = pd.read_csv(file_dir+f'{split}_converted.csv')
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(df[['user', 'movies', 'values']], reader)
    return data


In [11]:
from surprise import SVDpp, accuracy
train_data = load_dataset(train_path, 0)
test_data = load_dataset(val_path, 0)

algo = SVDpp()
trainset = train_data.build_full_trainset()
algo.fit(trainset)
testset = test_data.build_full_trainset()
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)

TypeError: 'Trainset' object is not iterable

In [16]:
testset = test_data.build_full_trainset().build_testset()
predictions = algo.test(testset)
accuracy.rmse(predictions, verbose=True)

RMSE: 1.0135


1.013488090573071

In [17]:
from surprise import dump
def train_split(log_dir, split = 0):
    train_data = load_dataset(train_path, split)
    test_data = load_dataset(val_path, split)

    algo = SVDpp(verbose=True, n_factors=50, n_epochs=30)
    trainset = train_data.build_full_trainset()
    algo.fit(trainset)
    testset = test_data.build_full_trainset().build_testset()
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)
    dump.dump(log_dir+f'/svdpp_default_{split}', algo=algo)

In [ ]:
log_dir = '/home/jimmy/CILProject22/models'
for i in range(0, 10):
    train_split(log_dir, i)

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
